# Heroes Of Pymoli Data Analysis
* Despite females being in the minority of players, female players average total purchases are nearly 10% higher than males

* The most items a single player has purchased to date is 5

* By average total purchases per player, the three highest spending age demographics are 35-39, <10, and 20-24
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
total_players=len(purchase_data['SN'].unique())
print(total_players)
# purchase_data.head()

576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# Total Unique Items
unique_items=len(purchase_data['Item Name'].unique())

# Average Purchase Price
avg_purch_price=purchase_data['Price'].mean()

# Total Purchases
total_purchases=len(purchase_data['Purchase ID'])

# Total Revenue
total_revenue=purchase_data['Price'].sum()

# Create Summary Data Frame
summary_df=pd.DataFrame({
    'Unique Items':[unique_items],
    'Average Purchase Price':avg_purch_price,
    'Total Purchases':total_purchases,
    'Total Revenue':total_revenue
})

summary_df['Average Purchase Price']=summary_df['Average Purchase Price'].map("${:,.2f}".format)
summary_df['Total Revenue']=summary_df['Total Revenue'].map("${:,.2f}".format)

print(summary_df)

   Unique Items Average Purchase Price  Total Purchases Total Revenue
0           179                  $3.05              780     $2,379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# Group by Gender & count unique SN & put results in data frame
gender_summary=pd.DataFrame(purchase_data.groupby(['Gender']).SN.nunique())
gender_summary['Percentage of Players']=round(gender_summary["SN"]/total_players*100,2)

# Rename Columns
gender_summary=gender_summary.rename(columns={'SN':'Total Count'})

# Order by frequency
gender_summary=gender_summary.sort_values(["Total Count"],ascending=False)

print(gender_summary)


                       Total Count  Percentage of Players
Gender                                                   
Male                           484                  84.03
Female                          81                  14.06
Other / Non-Disclosed           11                   1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# Create group by object by gender
gender_group=purchase_data.groupby(['Gender'])

# Purchase count
purchase_count=pd.DataFrame(gender_group['Purchase ID'].count())

# Average Purchase Price
avg_purchase_price=pd.DataFrame(gender_group['Price'].mean())

# Total Purchases
total_purchases=pd.DataFrame(gender_group['Price'].sum())

# Average Purchase Total By Person For Gender
# Group by Gender & SN; then find sum of price; then group by gender, then find the mean
mean_tot_purch=pd.DataFrame(purchase_data.groupby(['Gender','SN'])['Price'].sum().groupby(["Gender"]).mean())

# Create summary data frame
# Change indexes for each data frame
purchase_count.reset_index(inplace=True)
avg_purchase_price.reset_index(inplace=True)
total_purchases.reset_index(inplace=True)
mean_tot_purch.reset_index(inplace=True)

# Merge Tables
merge1=pd.merge(purchase_count,avg_purchase_price,on="Gender")
merge2=pd.merge(merge1,total_purchases,on="Gender",suffixes=(' avg',' total'))
merge3=pd.merge(merge2,mean_tot_purch,on="Gender")

# Clean Table Up
gender_purchase_analysis=merge3.rename(columns={
    'Purchase ID':'Purchase Count',
    'Price avg':'Average Purchase Price',
    'Price total':'Total Purchase Value',
    'Price':'Avg Total Purchases per Person'
})

# Format Numbers
gender_purchase_analysis["Average Purchase Price"]=gender_purchase_analysis["Average Purchase Price"].map("${:,.2f}".format)
gender_purchase_analysis["Total Purchase Value"]=gender_purchase_analysis["Total Purchase Value"].map("${:,.2f}".format)
gender_purchase_analysis["Avg Total Purchases per Person"]=gender_purchase_analysis["Avg Total Purchases per Person"].map("${:,.2f}".format)

print(gender_purchase_analysis)

                  Gender  Purchase Count Average Purchase Price  \
0                 Female             113                  $3.20   
1                   Male             652                  $3.02   
2  Other / Non-Disclosed              15                  $3.35   

  Total Purchase Value Avg Total Purchases per Person  
0              $361.94                          $4.47  
1            $1,967.64                          $4.07  
2               $50.19                          $4.56  


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# Establish bins for ages & labels

# Check min & max
purchase_data.describe()
age_bins=[0,9,14,19,24,29,34,39,100]
age_labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Use cut to categorize players
purchase_data["Age Bin"]=pd.cut(purchase_data["Age"],age_bins,labels=age_labels)

# Counts by age group
age_summary=pd.DataFrame(purchase_data.groupby(['Age Bin']).SN.nunique())

# Percentages by age group; round & create whole numbers
age_summary['Percent']=round(age_summary['SN']/total_players,4)*100

# Rename columns
age_summary=age_summary.rename(columns={'SN':'Total Players'})
print(age_summary)


         Total Players  Percent
Age Bin                        
<10                 17     2.95
10-14               22     3.82
15-19              107    18.58
20-24              258    44.79
25-29               77    13.37
30-34               52     9.03
35-39               31     5.38
40+                 12     2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Group by age bin
age_bin_summary=purchase_data.groupby(['Age Bin'])

# Purchase count
age_bin_count=pd.DataFrame(age_bin_summary["Purchase ID"].count())

# Average Purchase Price
age_bin_avg=pd.DataFrame(age_bin_summary['Price'].mean())

# Total Purchases
age_bin_total=pd.DataFrame(age_bin_summary['Price'].sum())

# Average Purchase Total per Person For Age Bin
mean_age_purch=pd.DataFrame(purchase_data.groupby(['Age Bin','SN'])['Price'].sum().groupby(['Age Bin']).mean())

# Create summary data frame
# Change indexes for each data frame
age_bin_count.reset_index(inplace=True)
age_bin_avg.reset_index(inplace=True)
age_bin_total.reset_index(inplace=True)
mean_age_purch.reset_index(inplace=True)

# Merge Tables
mergeA=pd.merge(age_bin_count,age_bin_avg,on="Age Bin")
mergeB=pd.merge(mergeA,age_bin_total,on="Age Bin",suffixes=(' avg',' total'))
mergeC=pd.merge(mergeB,mean_age_purch,on="Age Bin")

# Clean Table Up
age_bin_analysis=mergeC.rename(columns={
    'Purchase ID':'Purchase Count',
    'Price avg':'Average Purchase Price',
    'Price total':'Total Purchase Value',
    'Price':'Avg Total Purchase per Person'
})

# Format Numbers
age_bin_analysis['Average Purchase Price']=age_bin_analysis['Average Purchase Price'].map("${:,.2f}".format)
age_bin_analysis['Total Purchase Value']=age_bin_analysis['Total Purchase Value'].map("${:,.2f}".format)
age_bin_analysis['Avg Total Purchase per Person']=age_bin_analysis['Avg Total Purchase per Person'].map("${:,.2f}".format)


print(age_bin_analysis)

  Age Bin  Purchase Count Average Purchase Price Total Purchase Value  \
0     <10              23                  $3.35               $77.13   
1   10-14              28                  $2.96               $82.78   
2   15-19             136                  $3.04              $412.89   
3   20-24             365                  $3.05            $1,114.06   
4   25-29             101                  $2.90              $293.00   
5   30-34              73                  $2.93              $214.00   
6   35-39              41                  $3.60              $147.67   
7     40+              13                  $2.94               $38.24   

  Avg Total Purchase per Person  
0                         $4.54  
1                         $3.76  
2                         $3.86  
3                         $4.32  
4                         $3.81  
5                         $4.12  
6                         $4.76  
7                         $3.19  


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# Top Spenders - Purchase Count, Average Purchase Price, Total Purchase Value
player_group=purchase_data.groupby('SN')

# Purchase Count
player_purch_count=player_group['Purchase ID'].count()

# Average Purchase Price
player_avg_purch=player_group['Price'].mean()

# Total Purchase Value
player_total_purch=player_group['Price'].sum()

player_total_purch.head()

# Create summary dataframe
player_summary=pd.DataFrame({
    'Purchase Count': player_purch_count,
    'Average Purchase Price': player_avg_purch,
    'Total Purchase Value':player_total_purch
})

# sort values
player_summary=player_summary.sort_values(['Total Purchase Value'],ascending=False)

# Reformat data
player_summary["Average Purchase Price"]=player_summary["Average Purchase Price"].map("${:,.2f}".format)
player_summary["Total Purchase Value"]=player_summary["Total Purchase Value"].map("${:,.2f}".format)

player_summary.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
# Group by Item Name
item_group=purchase_data.groupby(['Item Name','Item ID'])

# Purchase Count
item_purch_count=item_group['Purchase ID'].count()
item_count_df=pd.DataFrame(item_purch_count)

# Average Purchase Price
item_avg_purch=item_group['Price'].mean()
item_avg_df=pd.DataFrame(item_avg_purch)

# Total Purchase Value
item_purch_total=item_group['Price'].sum()
item_total_df=pd.DataFrame(item_purch_total)

# Create summary dataframe
merge_1=pd.merge(item_count_df,item_avg_df,on=["Item Name",'Item ID'])
merge_2=pd.merge(merge_1,item_total_df,on=["Item Name", "Item ID"],suffixes=(' Avg',' Total'))

# Create summary dataframe
# Rename dataframe
item_summary=merge_2.rename(columns={
    "Purchase ID":'Purchase Count',
    "Price Avg":'Item Price',
    "Price Total":'Total Purchase Value'
})

# Sort by purchase count descending
item_summary=item_summary.sort_values(['Purchase Count'],ascending=False)

# Re-format
item_summary['Item Price']=item_summary['Item Price'].map("${:,.2f}".format)
item_summary['Total Purchase Value']=item_summary['Total Purchase Value'].map("${:,.2f}".format)

# Move indexes over to avoid display errors in github
item_summary.reset_index(inplace=True)

# item_summary.head()
item_summary.head()


,Item Name,Item ID,Purchase Count,Item Price,Total Purchase Value
0,"Oathbreaker, Last Hope of the Breaking Storm",178,12,$4.23,$50.76
1,"Extraction, Quickblade Of Trembling Hands",108,9,$3.53,$31.77
2,Nirvana,82,9,$4.90,$44.10
3,Fiery Glass Crusader,145,9,$4.58,$41.22
4,"Pursuit, Cudgel of Necromancy",19,8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [18]:
# Re-create data frame to undo formatting
item_profit_summary=merge_2.rename(columns={
    "Purchase ID":'Purchase Count',
    "Price Avg":'Item Price',
    "Price Total":'Total Purchase Value'
})


# Sort table by total purchase value descending
item_profit_summary=item_profit_summary.sort_values(['Total Purchase Value'],ascending=False)

# Reformat values
item_profit_summary['Item Price']=item_profit_summary['Item Price'].map("${:,.2f}".format)
item_profit_summary['Total Purchase Value']=item_profit_summary['Total Purchase Value'].map("${:,.2f}".format)

# Reset index to avoiud dislay problems in github
item_profit_summary.reset_index(inplace=True)

# Display preview
# item_profit_summary.head()
item_profit_summary.iloc[0:5,:]

,Item Name,Item ID,Purchase Count,Item Price,Total Purchase Value
0,"Oathbreaker, Last Hope of the Breaking Storm",178,12,$4.23,$50.76
1,Nirvana,82,9,$4.90,$44.10
2,Fiery Glass Crusader,145,9,$4.58,$41.22
3,Final Critic,92,8,$4.88,$39.04
4,Singed Scalpel,103,8,$4.35,$34.80
